In [0]:
import dlt
from pyspark.sql.functions import col, max as spark_max, sum as sum_spark, desc, round, current_date, datediff

@dlt.table(
    name="playtime",
    comment="dados da bigtables gerando uma tabela playtime"
)
def playtime_silver():
    df = dlt.read("steamdatabricks_workspace.gold.bigtable") \
            .filter(col('playtime_forever') > 0.00)


    df_playtime= df.select("appid", "playtime_forever", "rtime_last_played").distinct()

    df_max_dates = (
        df_playtime.groupBy("appid")
            .agg(spark_max("rtime_last_played").alias("ultima_data_jogada"))
    )

    df_filtrado = (
        df_playtime.join(df_max_dates,
                (df_playtime.appid == df_max_dates.appid) &
                (df_playtime.rtime_last_played == df_max_dates.ultima_data_jogada),
                "inner"
        )
        .select(df_playtime["*"])  
    ).orderBy(col("playtime_forever").desc())

    df_com_tempo_sem_jogar = (
        df_filtrado.withColumn(
            "time_without_play",
            datediff(current_date(), col("rtime_last_played")).cast("int")
        )
    )

    total_horas = df_com_tempo_sem_jogar.agg(sum_spark("playtime_forever").alias("total_horas")).collect()[0]["total_horas"]

    df_proporcao = df_com_tempo_sem_jogar.withColumn(
        "proporcao_tempo",
        round((col("playtime_forever") / total_horas) * 100, 2)
    )

    return df_proporcao